In [ ]:
from sagemaker.pytorch import PyTorchModel
import sagemaker
import boto3
from datetime import datetime
import time

In [ ]:
s3_client = boto3.client('s3')
session = sagemaker.Session()
region = session.boto_region_name
role = sagemaker.get_execution_role()

In [ ]:
!tar -czvf model.tar.gz code/ model.pt
print("Model data ready")

In [ ]:
bucket_name = 'beats-finetuning'
model_file = 'model.tar.gz'

In [ ]:
s3_client.delete_object(Bucket=bucket_name, Key=model_file)
s3_client.upload_file(model_file, bucket_name, model_file)

In [ ]:
model_name = "model.pt"
model_data = f"s3://{bucket_name}/model.tar.gz"

In [ ]:
model = PyTorchModel(entry_point = "inference.py",
# source_dir = "./code",
# image_uri='my-user.dkr.ecr.us-east-1.amazonaws.com/my-model:latest',
model_data = model_data,
framework_version = "1.12",
py_version = "py38",
env = {"MODEL_NAME": model_name},
role = role,
sagemaker_session = session
)

In [ ]:
### FROM ECR
# model.image_uri = "my-user.dkr.ecr.us-east-1.amazonaws.com/my-model:latest"

In [ ]:
INSTANCE_TYPE = "ml.m5.4xlarge"
ENDPOINT_NAME = "beats-pytorch-" + str(datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S-%f"))

In [ ]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=INSTANCE_TYPE,
    endpoint_name=ENDPOINT_NAME,
    content_type = 'audio/wav'
)

In [ ]:
start_time = time.time()
predictions = predictor.predict("baby.wav")
duration = time.time() - start_time
print(predictions)
print(duration)

In [ ]:
predictor.delete_endpoint(delete_endpoint_config=True)